### Import some modules

In [1]:
import cv2
import time

# Set up Camera access(es) ###
Check possible device ids using: ```ls -ltrh /dev/video*```

In [2]:
devices = {'usb-cam': cv2.VideoCapture(1)}

3 video devices have been initialised:
dict_keys(['cam 0', 'cam 1', 'cam 2'])


### Define Live Preview Window ###
Initiate the necessary modules

In [3]:
%gui qt5
from pyqtgraph.Qt import QtGui, QtCore
import pyqtgraph as pyqtgraph

# and some modules for button widgets
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

/home/Tobi/.opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Define a Preview **object class**

In [4]:
class Preview(object):   
    """Pop up a qt5 window for each working camera and stream the **live camera view** using pyqtgraph"""
    def __init__(self, device):
        
        # assign camera
        self.device = devices[device]
        self.device_name = device
        
        # define preview window
        self.window_title = "Preview"
        self.window = pyqtgraph.GraphicsWindow(title=self.window_title)
        self.window.resize(320,240)
        self.viewport = pyqtgraph.GraphicsLayout()
        self.view = self.viewport.addViewBox()
        self.window.setCentralItem(self.viewport)
        
        # init timer for data refresh
        self.stream = QtCore.QTimer()
        self.stream.timeout.connect(self.update)
        self.stream.start(100) # pass time in miliseconds
        
        # define button for saving images
        self.save_button = widgets.Button(description= '', button_style = '', icon='save')
        self.save_button.on_click(self.save)
        
        
    def update(self):
        bgr_img = [self.device.read() for i in range(7)][-1]
        b,g,r = cv2.split(bgr_img[1])
        rgb_img = cv2.merge([r,g,b])

        frame = pyqtgraph.ImageItem(rgb_img)
        self.view.addItem(frame)
        
    def save(self, b):
        self.stream.stop()

        img = [self.device.read() for i in range(7)][-1]
        cv2.imwrite(self.device_name +'_'+time.asctime()+'.png', img[1])

        self.stream.start(100)

**Note:** *Code for saving pictures explained*:
The camera is somewhat slow in data-acquisition, a changed view may not be reflected immediately in the picture. 
Therefore we send 7 read() requests and only store the last one (index [-1]) in the variable "img".
Each read() request returns a tuple of success (True, False) and the image data, the latter is written to a *".png"* file

# Live Preview#
Pop up a qt5 window for each working camera and stream the **live camera view** using pyqtgraph

In [7]:
preview = Preview('usb-cam')

# show the save-buttons of all devices    
display(preview.save_button)

HBox(children=(Button(description='cam 1', icon='save', style=ButtonStyle()), Button(description='cam 2', icon='save', style=ButtonStyle())))

## Movement Control## 
using *Micos Corvus Stage*

In [1]:
import os,serial as sl
from modules.MyStage import MyStage, MyAxis
Micos = MyStage(os, sl)
Micos.user_init()
Micos.set_acceleration(50)


 Stage module loaded - do "dmesg | grep tty" to list Ports of connected serial devices:

Port from output : 

 Try to connect to SerialPort: 
/dev/ttyUSB0
Successfuly connected to Micos Corvus Stage


In [2]:
ax = MyAxis(1, Micos)
ax.go_home()
Micos.set_home()

# Test movement with respect to camera vision #

In [3]:
ax.move_relative(45)

## **End the streams** and close the pop-up windows

In [10]:
preview.stream.stop()
preview.save_button.disable = True
preview.window.close()

# Start continuous picture acquisition#
Define period in minutes

In [6]:
period = 2

**Define the actions** you want to be taken after every period of time, e. g. save the last of several pictures from the camera to a file with a timestamp, then waits for the specified period until next repetition. 

In [1]:
def action():
    
    position1 = 'left'
    # move to first position
    ax.move_relative(-45)
    # save image
    img = [devices['usb-cam'].read() for i in range(7)][-1]
    cv2.imwrite(position1 + '_' + time.asctime()+'.png', img[1])

    # go back to initial position
    ax.go_home()

    position2 = 'right'
    # move to next position
    ax.move_relative(45)
    # save image
    img = [devices['usb-cam'].read() for i in range(7)][-1]
    cv2.imwrite(position2 +'_'+ time.asctime()+'.png', img[1])

    # finally, wait specified time
    time.sleep(period*60)  # pass period in seconds

NameError: name 'preview' is not defined

Start a **while loop**, which saves a picture for each camera, then waits for the specified period until next repetition. 
(* **Note**: repeats forever if you don't stop it, e.g. using the notebook's "interrupt Kernel" button above*)

In [1]:
while True:
    try: 
        action()
            
    except KeyboardInterrupt:
        print('Process ended by user.')
        break

NameError: name 'preview' is not defined